# FIGARO: skymaps

This notebook shows how to use FIGARO to reconstruct the skymap of a gravitational wave event given a set of posterior samples.

## Prerequisites

This notebook makes use of the PE samples from the various GWTC data release. FIGARO comes with GW150914 posterior samples in .hdf5 format. The GWTC data releases are available via [GWOSC](https://www.gw-openscience.org).

In the following we will make use of a preprocessed version of [GLADE+](http://glade.elte.hu).\
The catalog file is ~1.2 GB (the original .txt file is ~6 GB), therefore is not attached to FIGARO.\
Before running this notebook, please download GLADE+ from the website and preprocess it using the `create_glade.py` script , located in the `pipelines` folder: `python create_glade.py -i path/to/glade+.txt`. This will create, in the same folder as `glade+.txt`, `glade+.hdf5`.

Once done, please provide the location of your `glade+.hdf5` file: 

In [ ]:
glade_file = 'path/to/glade+.hdf5'

## GW150914

### Loading posterior samples

First of all, we need to load the posterior samples from the `.hdf5` file. FIGARO provides an ad-hoc module, `figaro.load`:

In [ ]:
from figaro.load import load_single_event
import numpy as np

samples, name = load_single_event('./data/GW150914_GWTC-1.hdf5', par = ['ra', 'dec', 'luminosity_distance'])

Please note the order of the parameters: the module we are going to use expects them to be in this precise order, $[\alpha,\delta,D_L]$.

Let's have a quick glance at the samples:

In [ ]:
from corner import corner

c = corner(samples, labels = ['$\\alpha$', '$\\delta$', '$D_L$'], color = 'dodgerblue')

The class we want to use to analyse these samples is `figaro.threeDvolume.VolumeReconstruction`:

In [ ]:
from figaro.threeDvolume import VolumeReconstruction

This class takes several parameters (only one of them is mandatory, though). Here we report the ones which are most useful for a turnkey use, postponing the others :
 * `max_dist`, **mandatory**: upper bound of the distance prior in Mpc. The other end is set, by default, to 0. Beware that setting `max_dist = max(samples[:,2])` is not allowed and raises an exception. This is intentional: FIGARO does not allow the user to determine parameter space bounds directly using samples values;
 * `out_folder`: folder where to save the outputs;
 * `n_gridpoints`: number of grid points for each parameter. Default is 720 for $\alpha$, 360 for $\delta$ ($0.25\ \mathrm{deg}^2$ resolution) and 100 for $D_L$. For a quicker run,  which however gives a less precise skymap, use smaller numbers;
 * `name`: event name, to be given to outputs;
 * `latex`: use LaTeX in plots (it takes a bit longer to produce outputs, but they are nicer).

In this example, the maximum distance will be set to 1000 Mpc. Instancing the `VolumeReconstruction` with default values takes around a minute because it needs to compute a 720x360x100 grid.

In [ ]:
d_max = 1000

mix = VolumeReconstruction(max_dist   = d_max,
                           out_folder = './skymap_test',
                           name       = name,
                           latex      = True,
                           )

`VolumeReconstruction` works identically to `figaro.mixture.DPGMM`. To reconstruct the skymap, just use the `density_from_samples` method:

In [ ]:
mix.density_from_samples(samples)

The skymap is stored in `out_folder/skymaps/name/`:

In [ ]:
from IPython.display import display_pdf
from pathlib import Path

with open(Path(mix.out_folder, 'skymaps/{0}/{0}_all.pdf'.format(mix.name)).resolve(), "rb") as f:
    display_pdf(f.read(),raw=True)